# sql_analysis.ipynb
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

This Jupyter Notebook analyzes the postgreSQL database.

The queries containe herin were developed in sql.sql and transformed into sqlalchemy.

In [13]:
# Import Dependencies
import sys
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

# Import database password
try:
    from postgres_pswd import host, database, username, passwd
    if host == 'YOUR DATABASE HOST HERE':
        sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
        from postgres_remote import host, database, username, passwd
except:
    sys.path.append(r"C:\Users\nlund\Documents\GitHub\untracked_files")
    from postgres_remote import host, database, username, passwd

# Create engine to mutual_funds database
engine_startup = 'postgresql://' + username + ":" + passwd + "@" + host + '/' + database
engine = create_engine(engine_startup)

In [14]:
# reflect the existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
Base.classes.keys()

['sp500', 'fund_holdings']